In [18]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf

from src.data.data_utils import Dataset

## Setup
- enable eager execution for TF for easier prototyping
- load train and test datasets
- define main architecutre params

In [26]:
tf.enable_eager_execution()
train_dataset = Dataset.load_dataset('../data/processed/train_dataset.pckl')
test_dataset = Dataset.load_dataset('../data/processed/test_dataset.pckl')

train_scans = [train_dataset.patients[p].scans for p in train_dataset.patient_ids]
test_scans = [test_dataset.patients[p].scans for p in test_dataset.patient_ids]